## Колокації

Одним із важливих аспектів будь-якої природної мови є сполучуваність слів. В англійській мові поняття сполучуваності дуже відчутне і присутнє у всіх мовних структурах: ми кажемо _"make a mistake",_ але _"do a favour";_ ми кажемо _"big surprise",_ але _"great anger";_ ми кажемо _"highly unlikely",_ але _"seriously wrong"._

У цьому завданні вам потрібно дослідити сполучуваність дієслів одного синонімного ряду з прислівниками. Наприклад, ми частіше кажемо _"love somebody dearly", "honor somebody highly",_ але _"admire somebody greatly"._

**Завдання:**
1. продовжте синонімний ряд дієслів: _"say", "tell", "speak", "claim", "communicate"_
2. напишіть функцію, яка знаходить у реченні дієслово (за складеним раніше синонімним рядом) і витягає усі можливі прислівники на "-ly", якими це дієслово керує
3. напишіть програму, яка знайде усі можливі прислівники для наших дієслів у [корпусі блогів](blog2008.txt)
4. на виході програма повинна видати десять найчастотніших прислівників для кожного дієслова (разом із частотою). Приклад виводу:
	```
	say: (loudly, 51), (silently, 45), (quietly, 10)
	tell: (quietly, 100), (loudly, 61), (seriously, 5)
	```
5. збережіть програму та результати аналізу на корпусі (вивід) у директорії з вашим іменем

Синоніми можна знайти у тезаурусах (http://www.thesaurus.com/, https://www.merriam-webster.com/thesaurus) чи [WordNet](http://wordnetweb.princeton.edu/perl/webwn).

Корпус блогів взятий з [Political Blog Corpora](http://www.cs.cmu.edu/~ark/blog-data/).

Ви можете використати будь-яку мову програмування та будь-яку NLP-бібліотеку.

P.S. Зверніть увагу на те, що у природній мові дієслова можуть мати різні форми, а прислівників може бути по декілька.

### 1. продовжте синонімний ряд дієслів: _"say", "tell", "speak", "claim", "communicate"_

In [1]:
from nltk.corpus import wordnet as wn

entry_verbs = ['say', 'tell', 'speak', 'claim', 'communicate']


def syns(verb):
    return {lemma.name().replace('_', ' ') \
            for synset in wn.synsets(verb, wn.VERB) for lemma in synset.lemmas()}
  

synonyms = {syn for verb in entry_verbs for syn in syns(verb)}

print(synonyms)

{'separate', 'speak', 'narrate', 'put across', 'sound out', 'verbalise', 'arrogate', 'intercommunicate', 'exact', 'lay claim', 'enunciate', 'assure', 'order', 'evidence', 'secern', 'convey', 'communicate', 'commune', 'suppose', 'talk', 'severalise', 'transmit', 'allege', 'tell apart', 'secernate', 'enjoin', 'pronounce', 'recount', 'mouth', 'take', 'pass', 'differentiate', 'enounce', 'state', 'articulate', 'aver', 'read', 'claim', 'distinguish', 'tell', 'severalize', 'utter', 'recite', 'verbalize', 'pass on', 'say', 'address', 'pass along'}


### 2. напишіть функцію, яка знаходить у реченні дієслово (за складеним раніше синонімним рядом) і витягає усі можливі прислівники на "-ly", якими це дієслово керує

In [2]:
import spacy

nlp = spacy.load("en_core_web_md", disable = ['ner'])

In [3]:

def reach(from_token, to_verb):
    if from_token.head.lemma_ == to_verb and from_token.head.pos_ == 'VERB':
        return from_token.head
    elif from_token.dep_ == 'ROOT':
        return None
    
    return reach(from_token.head, to_verb)
    
def extract_deply_adverbs(doc, verb):
    
    parts = verb.split(' ')
    parts_num = len(parts)
    
    if not parts_num or parts_num > 2:
        raise ValueError("Incorrect verb parts number")

    adverbs = set([])
    
    for token in doc:
        if token.text.endswith('ly') and token.pos_ == 'ADV':
            head = reach(token, parts[0])
            if head:
                if (parts_num == 2):
                    if list(filter(lambda x: x.pos_ == 'PART' and x.text == parts[1], head.children)):
                        adverbs.add(token.text)
                else:
                    adverbs.add(token.text)
    
    return adverbs
    
    
assert extract_deply_adverbs(nlp("He speaks silently all the time"), "speak") == {'silently'}
assert extract_deply_adverbs(nlp("He speaks very silently all the time"), "speak") == {'silently'}
assert extract_deply_adverbs(nlp("He speaks loudly and seriously"), "speak") == {'loudly', 'seriously'}
assert extract_deply_adverbs(nlp("He speaks very loudly all the time and quite seriously"), "speak") == {'loudly', 'seriously'}
assert extract_deply_adverbs(nlp("He put accross loudly a few words"), "put accross") == set(['loudly'])
assert not extract_deply_adverbs(nlp("He put a letter in mailbox located accross the street"), "put accross")
assert not extract_deply_adverbs(nlp("He put a hat on"), "put accross")
assert not extract_deply_adverbs(nlp("He speaked a lot"), "speak")
assert not extract_deply_adverbs(nlp("He speaked loudly"), "tell")
    

### 3. напишіть програму, яка знайде усі можливі прислівники для наших дієслів у [корпусі блогів](blog2008.txt)

In [4]:
from collections import defaultdict

def word_hist(words):
    hist = defaultdict(int)
    for w in words:
        hist[w] += 1
    return dict(hist)

In [ ]:
stats = defaultdict(list)

with open('blog2008.txt') as sentences:
    for i, sentence in enumerate(sentences):
        #if i > 1000:
        #    break
        doc = nlp(sentence)
        for syn in synonyms:
            stats[syn].extend(extract_deply_adverbs(doc, syn))

hist = {}

for syn, adverbs in stats.items():
    if adverbs:
        hist[syn] = word_hist(adverbs)

In [24]:
for w, h in sorted(hist.items(), key = lambda kv: kv[0]):
    print("{}: {}".format(w, sorted(h.items(), key = lambda kv: kv[1], reverse = True)[0:10]), end = '\n\n')

address: [('directly', 17), ('only', 13), ('actually', 8), ('really', 7), ('publicly', 5), ('adequately', 3), ('recently', 3), ('immediately', 3), ('mostly', 3), ('effectively', 3)]

allege: [('potentially', 4), ('repeatedly', 3), ('subsequently', 3), ('personally', 2), ('flatly', 2), ('only', 2), ('wrongly', 2), ('directly', 2), ('unlawfully', 1), ('credibly', 1)]

articulate: [('clearly', 4), ('proactively', 1), ('fully', 1), ('astutely', 1), ('convincincly', 1), ('certainly', 1), ('really', 1), ('simply', 1)]

assure: [('privately', 4), ('only', 3), ('really', 3), ('apparently', 2), ('simply', 2), ('virtually', 2), ('falsely', 2), ('actually', 2), ('fully', 2), ('immediately', 2)]

claim: [('only', 64), ('falsely', 49), ('actually', 22), ('really', 19), ('Falsely', 17), ('simply', 16), ('nearly', 12), ('repeatedly', 12), ('merely', 11), ('previously', 9)]

communicate: [('directly', 4), ('effectively', 3), ('freely', 1), ('Really', 1), ('verbally', 1), ('indirectly', 1), ('apparentl